In [1]:
!pip install pyDOE
!pip install cma


  Created wheel for pyDOE: filename=pyDOE-0.3.8-cp36-none-any.whl size=18178 sha256=c34b6616736261dae8f3ef3041691ff88de94bcef1cbc02703ce20aee3eda236
  Stored in directory: /root/.cache/pip/wheels/7c/c8/58/a6493bd415e8ba5735082b5e0c096d7c1f2933077a8ce34544
Successfully built pyDOE
     |████████████████████████████████| 245kB 2.7MB/s 


In [0]:
import pyDOE
from scipy import stats
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import Bounds
from collections import namedtuple
import cma
from sklearn.preprocessing import MinMaxScaler
import scipy.stats.distributions as dist
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.gaussian_process.kernels import DotProduct
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.linear_model import ElasticNet
from sklearn.preprocessing import PolynomialFeatures
import bbobbenchmarks as bn
import sklearn

## Surrogate Models & Helper Functions

In [0]:
ValueRange = namedtuple('ValueRange', ['min', 'max'])

def determinerange(values):
    """Determine the range of values in each dimension"""
    return ValueRange(np.min(values, axis=0), np.max(values, axis=0))


def linearscaletransform(values, *, range_in=None, range_out=ValueRange(0, 1), scale_only=False):
    """Perform a scale transformation of `values`: [range_in] --> [range_out]"""

    if range_in is None:
        range_in = determinerange(values)
    elif not isinstance(range_in, ValueRange):
        range_in = ValueRange(*range_in)

    if not isinstance(range_out, ValueRange):
        range_out = ValueRange(*range_out)

    scale_out = range_out.max - range_out.min
    scale_in = range_in.max - range_in.min

    if scale_only:
        scaled_values = (values / scale_in) * scale_out
    else:
        scaled_values = (values - range_in.min) / scale_in
        scaled_values = (scaled_values * scale_out) + range_out.min

    return scaled_values


''' F10 '''
def F10(X):
    f = bn.F10()
    X = np.array(X)
    return f(X)

''' Latin HyperCube Sampling Design of Experiment '''
def DOE(n_obs, dim):
    np.random.seed(0)
    lhd = pyDOE.lhs(n=dim, samples=n_obs, criterion='m')
    X = [lhd[:,idx] for idx in range(dim)]
    return X

def create_basis_function(data):
    true = np.array(data['Y'])
    data = pd.DataFrame(np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(data.iloc[:,:-1])))
    data['Y'] = pd.Series(true)
    return data

''' Create Basis Functions '''
def create_function_basis(x):
    return np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(x.reshape(1,-1)))
      

''' Elastic Net Regression '''
def elastic_net(train_data,test_data):
    scaler =  MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    regr = ElasticNet(alpha= 791.39 ,random_state=0 , l1_ratio= 1.0, fit_intercept =True, max_iter=3000,selection='random').fit(scaler.transform ( np.array(train_data.iloc[:,:-1])) ,  np.array(train_data.iloc[:,-1]))
    pred = regr.predict(scaler.transform(test_data))
    def predict(scaler, regr):
        def __predict__(x):
            x = create_function_basis(x)
            return regr.predict(scaler.transform(x))
        return __predict__
    return regr,pred, predict(scaler, regr)
    

''' Kriging'''
def kriging(train_data,test_data):
    kernel =  RBF()
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = GaussianProcessRegressor(kernel=kernel,n_restarts_optimizer= 15,random_state=0,
                                   normalize_y=True ).fit(scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)

''' Support Vector Regression'''
def _SVR(train_data,test_data):
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = sklearn.svm.SVR(kernel='rbf', gamma = 4.96 , C = 1000.000000 ,max_iter=1500).fit( scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)

## Load Training and Test Data Set initially Generated

In [0]:
path = "train_10_1000Samples.csv"
train_y = pd.read_csv(path).iloc[:,-1]
test_y = pd.read_csv('test_10_200Samples.csv').iloc[:,-1]
path = "train_latent_50D.csv"
train = pd.read_csv(path, index_col = 0)
test = pd.read_csv('test_latent_50D.csv', index_col = 0)
train ['Y'] = train_y
test ['Y'] = test_y
del train_y
del test_y
true = np.array(test['Y'])

## Surrogate Models

In [5]:
model_kri , pred_kri , predict_kri = kriging(train,test.iloc[:,:-1])
model_svr , pred_svr , predict_svr = _SVR(train,test.iloc[:,:-1])
train = create_basis_function(train)
test = create_basis_function(test)
model_eln , pred_eln , predict_eln = elastic_net(train,test.iloc[:,:-1])

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 625690654205856.0, tolerance: 38292438908020.56
  positive)


## CMA-ES

In [6]:
Columns = ['Kri' , 'SVR' , 'ELN' ]
Cols = []
for j in range(len(Columns)):
    for i in range(1,26):
        Cols.append(Columns[j]+'_Z'+str(i))

opt = cma.CMAOptions()
opt.set ("seed" , 0)
opt.set ("maxfevals" , 25000)

n_obs , dim =  30, 25
G = DOE(n_obs, dim)
G = [ G[idx].reshape(n_obs,1) for idx in range(len(G)) ]
X_Values = np.zeros([30,75])
for i in range(G[1].shape[0]):
    print ('Run : '+ str(i))
    min_kri = cma.fmin(predict_kri , np.concatenate(G, 1)[i] , 0.1278 , options=opt) [0]
    min_svr = cma.fmin(predict_svr , np.concatenate(G, 1)[i] , 0.1278 , options=opt) [0]
    min_eln = cma.fmin(predict_eln , np.concatenate(G, 1)[i] , 0.1278 , options=opt) [0]
    X_Values [i,:] = list(min_kri)+list(min_svr)+list(min_eln)

Run : 0
(6_w,13)-aCMA-ES (mu_w=4.0,w_1=38%) in dimension 25 (seed=899905, Mon Apr 13 13:03:40 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     13 4.205033066956758e+07 1.0e+00 1.22e-01  1e-01  1e-01 0:00.0
    2     26 4.168102658783277e+07 1.1e+00 1.20e-01  1e-01  1e-01 0:00.0


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [42146430.85261746] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [42092688.32384511] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


    3     39 4.162450978284369e+07 1.1e+00 1.22e-01  1e-01  1e-01 0:00.0
  100   1300 -7.219055520973936e+06 1.8e+00 8.55e-03  7e-03  8e-03 0:00.9
  200   2600 -7.343727271866515e+06 2.0e+00 1.93e-04  1e-04  2e-04 0:01.8
  300   3900 -7.343767390280254e+06 2.3e+00 2.48e-06  1e-06  2e-06 0:02.8
  400   5200 -7.343767394121870e+06 2.5e+00 3.73e-08  2e-08  2e-08 0:03.7
  500   6500 -7.343767394122891e+06 2.7e+00 1.87e-09  8e-10  1e-09 0:04.6
  600   7800 -7.343767394122899e+06 3.7e+00 1.26e-09  4e-10  8e-10 0:05.5
  700   9100 -7.343767394122906e+06 4.6e+00 6.93e-10  2e-10  4e-10 0:06.4
  800  10400 -7.343767394122891e+06 5.9e+00 1.37e-09  5e-10  9e-10 0:07.3
  900  11700 -7.343767394122884e+06 7.3e+00 5.44e-10  2e-10  4e-10 0:08.2
 1000  13000 -7.343767394122884e+06 8.0e+00 3.84e-10  1e-10  3e-10 0:09.2
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
 1100  14300 -7.343767394122884e+06 9.5e+00 2.44e-10  8e-11  2e-10 0:10.1
 1200  15600 -7.343767394122884e+06 1.1e+

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [39119706.79597023] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [39119699.32185453] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


  100   1300 3.911234841730326e+07 1.7e+00 8.97e-03  8e-03  9e-03 0:00.8
  200   2600 3.911232705600537e+07 1.8e+00 1.61e-04  1e-04  1e-04 0:01.6
  300   3900 3.911232705306779e+07 2.1e+00 1.88e-06  1e-06  1e-06 0:02.3


...

  400   5200 3.911232705306738e+07 2.3e+00 1.05e-07  5e-08  7e-08 0:03.1
  419   5447 3.911232705306738e+07 2.4e+00 7.82e-08  3e-08  5e-08 0:03.2
termination on tolfun=1e-11 (Mon Apr 13 13:14:50 2020)
termination on tolfunhist=1e-12 (Mon Apr 13 13:14:50 2020)
termination on flat fitness=please (re)consider how to compute the fitness more elaborately if sigma=7.82e-08 is large (Mon Apr 13 13:14:50 2020)

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=39112327.053067, sigma=9.62e-08).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=404)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=39112327.053067, sigma=9.55e-08).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=407)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=39112327.053067, sigma=8.92e-08).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=410)
  ')')
/usr/local/li


final/bestever f-value = 3.911233e+07 3.911233e+07
incumbent solution: [ 0.11698448  0.13068148  0.08470866 -0.16325689 -0.15440351  0.18082949
  0.03821258  0.12865366 ...]
std deviations: [4.54926950e-08 4.20897266e-08 4.04422258e-08 5.25455923e-08
 3.83267533e-08 4.93260213e-08 4.62749513e-08 5.09220870e-08 ...]
(6_w,13)-aCMA-ES (mu_w=4.0,w_1=38%) in dimension 25 (seed=936886, Mon Apr 13 13:14:51 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     13 -2.026725629051502e+08 1.0e+00 1.19e-01  1e-01  1e-01 0:00.0
    2     26 -2.738181875027673e+08 1.1e+00 1.16e-01  1e-01  1e-01 0:00.0
    3     39 -2.955926472841731e+08 1.1e+00 1.14e-01  1e-01  1e-01 0:00.0


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [-80708203.2319592] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [-2.34520783e+08] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


   71    923 -1.993500492804951e+14 1.6e+00 1.37e+02  1e+02  1e+02 0:00.8
termination on tolfacupx=1000.0 (Mon Apr 13 13:14:51 2020)
final/bestever f-value = -1.927455e+14 -1.993500e+14
incumbent solution: [ 153.4826669   185.89652016  368.09449026  134.84814976  470.50787701
  260.70392416 -188.334357   -640.11261825 ...]
std deviations: [121.93712319 124.10816788 128.2753562  129.21069791 125.01287549
 124.35933404 134.36733708 132.47296288 ...]


In [7]:
X_Values = pd.DataFrame(X_Values)
X_Values.columns = Cols
X_Values.to_csv('F10_X_Values.csv')
Krig_Fun = np.zeros(30)
SVM_Fun = np.zeros(30)
ELN_Fun = np.zeros(30)
for i in range(X_Values.shape[0]):
    Krig_Fun [i] = F10(X_Values.iloc[i,:25])
    SVM_Fun [i] = F10(X_Values.iloc[i,25:50])
    ELN_Fun [i] = F10(X_Values.iloc[i,50:75])
print ('Kriging')
print (stats.mode(Krig_Fun) , np.std(Krig_Fun))
print ('SVM')
print (stats.mode(SVM_Fun) , np.std(SVM_Fun))
print ('ELN')
print (stats.mode(ELN_Fun) , np.std(ELN_Fun))

Kriging
ModeResult(mode=array([10175543.64979318]), count=array([1])) 380799.54057659244
SVM
ModeResult(mode=array([11455218.02092955]), count=array([1])) 1.1604520712582247
ELN
ModeResult(mode=array([4.23351912e+10]), count=array([1])) 127871547852.65147
